In [120]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
from pydantic import BaseModel
from fastapi import Request

class UserForm(BaseModel):
    username: str | None
    password: str | None


class UserFormValidation(BaseModel):
    valid_username: bool
    valid_password: bool

    @property
    def valid(self) -> bool:
        return True if self.valid_username and self.valid_password else False


In [15]:
form = UserForm(username=None, password="test")

In [17]:
if form.username:
    print(1)
else:
    print(2)

2


In [19]:
v = UserFormValidation(valid_password=True, valid_username=False)

In [20]:
v.dict()

{'valid_username': False, 'valid_password': True}

In [25]:
from passlib.context import CryptContext
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
pwd_context.hash("password2")

'$2b$12$6sLpSXOp7ujpe7eehqSmbeXMyDJB50zKiJv7fELWXob7zANWtnP/C'

In [26]:
len("$2b$12$6sLpSXOp7ujpe7eehqSmbeXMyDJB50zKiJv7fELWXob7zANWtnP/C")

60

In [50]:
class SignUpFormValidation(BaseModel):
    valid_username: bool
    valid_password: bool
    valid_adress: bool
    valid_city: bool
    valid_country: bool

    @property
    def valid(self) -> bool:
        bool_values_to_check = [
            self.valid_username,
            self.valid_password,
            self.valid_adress,
            self.valid_city,
            self.valid_country,
        ]
        return True if all(bool_values_to_check) else False
    
    def update_value(self, key: str, value: bool) -> None:
        setattr(self, key, value)


In [51]:
val = SignUpFormValidation(
    valid_adress=True,
    valid_city=True,
    valid_country=True,
    valid_password=True,
    valid_username=True,
)


In [56]:
val

SignUpFormValidation(valid_username=True, valid_password=True, valid_adress=False, valid_city=True, valid_country=True)

In [54]:
val.update_value("valid_adress", False)

In [55]:
val

SignUpFormValidation(valid_username=True, valid_password=True, valid_adress=False, valid_city=True, valid_country=True)

In [124]:
from src.sqlite import get_database_cursor_and_commit
from src.sqlite.models import UserInfo
from src.auth.models import SignUpForm, SignUpFormValidation

In [127]:
class UserInfo(BaseModel):
    user_id: int
    adress: str
    city: str
    country: str

In [125]:
new_user = "user1"
new_password = "password1"

In [137]:
USER_CREATE_QUERY = f"""
    --sql
    INSERT INTO users(username, password_hash)
    VALUES ('{new_user}', '{new_password}')
    RETURNING id;
    """
USER_INFO_CREATE_QUERY = """
    --sql
    INSERT INTO users_info(user_id, adress, city, country)
    VALUES (:user_id, :adress, :city, :country);
"""


In [101]:
def dict_from_row(row):
    return dict(zip(row.keys(), row))

In [134]:
form = SignUpForm(
    city="dadaad",
    username="aaa",
    password="aa",
    adress="asdads",
    country="dasasd",
)

In [138]:
with get_database_cursor_and_commit() as cursor:
    cursor.execute(USER_CREATE_QUERY)
    
    user_id = cursor.fetchone()[0]
    user = UserInfo(user_id=user_id, **form.dict())
    print(user)
    cursor.execute(USER_INFO_CREATE_QUERY, user.dict())
    
    # data = dict_from_row(cursor.fetchone())

user_id=20 adress='asdads' city='dadaad' country='dasasd'
